In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
import pickle

In [7]:
df = pd.read_csv('application_train.csv')

In [8]:
y_train = df.TARGET
df.drop(columns=['TARGET'], inplace=True)

In [9]:
columns_to_use = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_FAMILY_STATUS',  
           'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'DAYS_BIRTH']

In [10]:
df = df.loc[:, columns_to_use]

In [11]:
y_train = y_train[df.NAME_FAMILY_STATUS != 'Unknown']
df = df[df.NAME_FAMILY_STATUS != 'Unknown']

In [12]:
df.CODE_GENDER = df.CODE_GENDER.apply(lambda x: 1 if x == 'M' else 0)

In [13]:
df.FLAG_OWN_CAR = df.FLAG_OWN_CAR.apply(lambda x: 1 if x == 'Y' else 0)
df.FLAG_OWN_REALTY = df.FLAG_OWN_REALTY.apply(lambda x: 1 if x == 'Y' else 0)

In [14]:
ohe = OneHotEncoder(categories='auto', sparse=False)
family_status = ohe.fit_transform(df[['NAME_FAMILY_STATUS']])
family_status_df = pd.DataFrame(data=family_status, index = df.index, 
                                columns=df.NAME_FAMILY_STATUS.value_counts().index, dtype = int)
df.drop(columns=['NAME_FAMILY_STATUS'], inplace=True)

In [15]:
df = pd.concat([df, family_status_df], axis=1)

In [16]:
df.AMT_CREDIT = np.log(df.AMT_CREDIT)
df.AMT_INCOME_TOTAL = np.log(df.AMT_INCOME_TOTAL)

In [17]:
df.DAYS_BIRTH = df.DAYS_BIRTH.apply(lambda x: np.log1p(abs(x) / 365))

In [18]:
for _ in range(4):
    df = pd.concat([df, df[y_train == 1]], axis = 0)
    y_train = pd.concat([y_train, y_train[y_train == 1]], axis = 0)

In [23]:
result = pd.concat([df, y_train], axis = 1)

In [25]:
result.to_csv('train.csv')